In [15]:
# Import the required libraries and dependencies
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import sqlalchemy

# Add viz libs
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import hvplot.pandas
import altair as alt 
from bokeh.plotting import figure, show
import folium

## Clean data

In [16]:
# Force clean vars
force_clean_large_kickstarter = False
force_clean_small_kickstarter = False
force_clean_indiegogo = False

# Clean large kickstarter
my_kickstarter_large_file = Path('./Resources/kickstarter_data_clean/ks-projects-large.csv')
if not my_kickstarter_large_file.is_file() or force_clean_large_kickstarter:
    %run ./clean_kickstarter_large.ipynb

# Clean small kickstarter
my_kickstarter_small_file = Path('./Resources/kickstarter_data_clean/ks-projects-small.csv')
if not my_kickstarter_small_file.is_file() or force_clean_small_kickstarter:
    %run ./clean_kickstarter_small.ipynb

# Clean indiegogo kickstarter
my_indiegogo_file = Path('./Resources/indiegogo_data_clean/indiegogo-projects.csv')
if not my_indiegogo_file.is_file() or force_clean_indiegogo:
    %run ./clean_indiegogo.ipynb

## Enable Mapbox API access token

In [17]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv('MAPBOX_API_ACCESS_TOKEN')

# Confirm the availability of your Mapbox API access token by checking its type
display(type(mapbox_api_access_token))

# Set your Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

str

## Read kickstarter large

In [18]:
# Import small kickstarter most backed dataset
kickstarter_large_clean_df = pd.read_csv(
    Path('./Resources/kickstarter_data_clean/ks-projects-large.csv')
)

# Fix dates
kickstarter_large_clean_df['launched'] = pd.to_datetime(kickstarter_large_clean_df['launched'])
kickstarter_large_clean_df['deadline'] = pd.to_datetime(kickstarter_large_clean_df['deadline'])

# View head
kickstarter_large_clean_df.head(2)

,ID,name,main_category,currency,usd_goal_real,usd_pledged_real,deadline,launched,state,funded_percent,duration,daily_goal,daily_pledged,country,avg_backer_per_day,pledged_per_person
0,1000002330,The Songs of Adelaide & Abullah,Publishing,GBP,1533.95,0.0,2015-10-09,2015-08-11 12:12:28,0,0.0000,58,26.45,0.00,GB,0.00,0.0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,USD,30000.00,2421.0,2017-11-01,2017-09-02 04:43:57,0,0.0807,59,508.47,41.03,US,0.25,161.4


## Read kickstarter small

In [19]:
# Import small kickstarter most backed dataset
kickstarter_small_clean_df = pd.read_csv(
    Path('./Resources/kickstarter_data_clean/ks-projects-small.csv')
)

# View head
kickstarter_small_clean_df.head(2)

,ID,name,main_category,currency,usd_goal_real,usd_pledged_real,location,backers,num.backers.tier,pledge.tier
0,0,Exploding Kittens,Tabletop Games,USD,10000.0,8782571.0,"Los Angeles, CA",219382,"[15505, 202934, 200, 5]","[20.0, 35.0, 100.0, 500.0]"
1,1,Fidget Cube: A Vinyl Desk Toy,Product Design,USD,15000.0,6465690.0,"Denver, CO",154926,"[788, 250, 43073, 21796, 41727, 21627, 12215, ...","[1.0, 14.0, 19.0, 19.0, 35.0, 35.0, 79.0, 79.0..."


## Read indiegogo

In [20]:
# Import indiegogo dataset
indiegogo_clean_df = pd.read_csv(
    Path('./Resources/indiegogo_data_clean/indiegogo-projects.csv')
)

# Fix dates
indiegogo_clean_df['launched'] = pd.to_datetime(indiegogo_clean_df['launched'])
indiegogo_clean_df['deadline'] = pd.to_datetime(indiegogo_clean_df['deadline'])

# View head
indiegogo_clean_df.head(2)

,ID,name,main_category,currency,usd_goal_real,usd_pledged_real,deadline,launched,state,funded_percent,duration,daily_goal,daily_pledged,country
0,3936,Join the Electric Revolution!!!,Transportation,USD,5000.0,840.0,2010-05-12 23:59:00,2010-04-21 22:38:42,0,0.1680,21,238.10,40.00,US
1,5109,Relief Trip to Haiti,Human Rights,USD,1200.0,250.0,2010-07-02 23:59:00,2010-06-10 17:47:35,0,0.2083,22,54.55,11.36,US


## Add SQL database and simple functions to load and access tables in that database

In [35]:
# Establishes Database Connection with a temporary SQL db (we can update to give it a name later)
database_connection_string = "sqlite:///"

engine = sqlalchemy.create_engine(database_connection_string)

# Function to load table into DB
# data is the dataframe we want to save, 
# table name is the name of the new table (as a string value), 
# and engine is the engine input established earlier
def new_table(data, table_name):
    data.to_sql(f"{table_name}", engine, index=True, if_exists="replace")


# Lets us load the table of our choice from the database, just set the function equal to a new dataframe variable and run 
# must set the table name as a string value
def load_full_table(table_name):
    new_df = pd.read_sql_table(f"{table_name}", con=engine )
    return new_df

new_table(kickstarter_large_clean_df, "kickstarter_large")
new_table(kickstarter_small_clean_df, "kickstarter_small")
new_table(indiegogo_clean_df, "indie_gogo")

In [47]:
# Create a SQL query to get the main_category and duration of the Kickstarter Large dataframe.
query = """
SELECT main_category, duration
FROM kickstarter_large
"""

# This will let us read the query we applied earlier to create a dataframe.
kickstarter_large_duration_dataframe = pd.read_sql_query(
    query, 
    con= engine)
kickstarter_large_duration_dataframe

,main_category,duration
0,Publishing,58
1,Film & Video,59
2,Film & Video,44
3,Music,29
4,Food,34
...,...,...
331548,Food,29
331549,Film & Video,26
331550,Film & Video,45
331551,Technology,30


In [93]:
# Create a SQL query to get the main_category and 
# duration of the Kickstarter Large dataframe and 
# group them by main_category and get its average duration days.
query="""SELECT
   main_category, AVG(duration) AS average_duration_days
FROM
  kickstarter_large
GROUP BY
  main_category
ORDER BY 
    AVG(duration) DESC,
    main_category DESC;
"""
# This will let us read the query we applied earlier to create a dataframe.
kickstarter_large_groupby_maincategory_df = pd.read_sql_query(
    query, 
    con= engine)
kickstarter_large_groupby_maincategory_df


,main_category,average_duration_days
0,Music,34.304318
1,Technology,34.263891
2,Film & Video,34.009856
3,Journalism,33.250482
4,Design,33.138903
5,Food,32.896073
6,Comics,32.846629
7,Publishing,32.728632
8,Photography,32.514914
9,Theater,32.313122


In [95]:

ks_lrg_duration_plot = kickstarter_large_groupby_maincategory_df.hvplot(kind='bar',
                                            x='main_category',
                                            ylim=(30.0,35.0),
                                            rot=60,
                                            title="Kickstarter's Average Duration Days of Each Main Category",
                                            hover_color= 'orange'
                                            )

:Bars   [main_category]   (average_duration_days)